# GPELab simulation for Rabi coupled spin mixture BEC
I want to find the ground state for the BEC spin mixture in cigar trap with Rabi coupling. <br>

GPELab solves GPE equations like:
The GPE is $$i\hbar\frac{\partial \Psi}{\partial t} = \frac{\delta E}{\delta \Psi^*}(\Psi) = \left(-\frac{\hbar^2}{2m}\Delta + V(t,x)+(N-1) U_0|\Psi|^2\right)\Psi$$

with $U_0 = 4\pi\hbar^2 a_s /m$ the interaction strength between 2 particles, and the $(N-1)$ multiplies it because in a BEC with N bosons, each particle interacts with $N−1$ other particles (usually $N \sim N-1$ is written). 

The wavefunction is normalized $$\int |\Psi(t,x)|dx = 1 $$.

The energy functonal is $$E(\Psi) = \int \left[\frac{\hbar^2}{2m} |\nabla\Psi|^2 + V|\Psi|^2 + \frac{NU_0}{2}|\Psi|^4\right]dx $$. 

In that case we can observe how the prefactor $\frac{NU_0}{2}$ derives from the fact that the interaction energy is given by the sum over all the pairs of particles $E_{int} = \sum_{i<j} U_{0} \delta(x_i+x_j)$. Since the number of pairs is $(N-1)/2$, and $N-1 \sim N$, converting the sum into integral we have $$ E_{int} = \frac{U_0 N}{2}\int d^3x |\Psi|^4$$


## ADIMENSIONALIZATION of GPE

Let's consider the two component BEC and its GPE:
$$i\hbar \partial_t \begin{pmatrix} \Psi_1 \\ \Psi_2 \end{pmatrix} = \left[
-\frac{\hbar^2}{2m} \Delta + V_{ext} + \hbar/2 \begin{pmatrix} \delta & \Omega \\ \Omega & -\delta \end{pmatrix} +
\begin{pmatrix} Ng_{11}|\Psi_1|^2+Ng_{12}|\Psi_2|^2 & 0 \\ 0 & Ng_{22}|\Psi_2|^2+Ng_{12}|\Psi_1|^2 \end{pmatrix}
\begin{pmatrix} \Psi_1 \\ \Psi_2 \end{pmatrix}
\right] $$

one has to chose for some characterictic lengths:
$$T = \Omega \quad L = \sigma_r $$

where for $\sigma_r$ we may choose $\sigma_r=\sqrt{\frac{\hbar}{m\omega_r}} \rightarrow \sqrt{1/\omega_r}$ if we consider an hapmonic profile fro the BEC.

so that we can define the adimensional quantities:

$$\tilde{t} = t\Omega \quad \tilde{x} = x/a_r \quad \tilde{\Psi} = \Psi a_r^{3/2}$$

Applying this substitution of variables, the GPE for one of the two BEC components is:
$$i\hbar \frac{\partial (\tilde{\Psi}_1 a_r^{3/2})}{\partial(\tilde{t}/\Omega)} = \left( -\frac{\hbar^2}{2m} \Delta \frac{1}{(\tilde{\mathbf{x}}a_r)^2} + \frac{1}{2}m(\bar{\omega}\mathbf{\tilde{x}} a_r)^2 +\frac{\hbar}{2} (\delta + \Omega) + N(g_{11} (a_r)^{-3}|\tilde{\Psi}_1 |^2+g_{12} (a_r)^{-3}|\tilde{\Psi}_2 |^2)\right)\tilde{\Psi}_1 a_r^{3/2}$$

$$i \frac{\partial (\tilde{\Psi}_1)}{\partial\tilde{t}} = \left( -\frac{\hbar}{2m\Omega} \Delta \frac{1}{(\tilde{\mathbf{x}}a_r)^2} + \frac{1}{2}\frac{\bar{\omega}}{\Omega}(\mathbf{x})^2 +\frac{1}{2} \frac{(\delta + \Omega)}{\Omega} + \frac{N}{\hbar\Omega}(g_{11} (a_r)^{-3}|\tilde{\Psi}_1|^2+g_{12} (a_r)^{-3}|\tilde{\Psi}_2|^2)\right)\tilde{\Psi}_1$$

we define the dimensionless coefficients:
$$\alpha = \frac{\hbar}{2m\Omega a_r^2}$$
$$\gamma_{ii} = \frac{Ng_{ii}}{\hbar\Omega}\frac{1}{a_r^3}$$
$$\beta_r = \omega_r/\Omega$$

At that point we can also observe how all the ratios $\beta/\alpha$, $\gamma/\alpha$, $\gamma/\beta$... are indipendent from $\hbar$ and $m$. We put so $\hbar=1$ and $m = 1$.<br>

The characteristic dimensions for the system are so: time $T$, length $L$ and the energy $E_0$. <br>
$$T =1/\Omega; \quad L = 1/\sqrt{\omega_r}; \quad E_0 = \Omega$$ 
In that case the adimensional quantities are 
$$\tilde{\Omega} = 1;\quad \tilde{\omega_r} = \omega_r/\Omega$$

$$\tilde{a_z} = a_z / L = \sqrt{\omega_r/\omega_z}; \quad \tilde{a_r} = a_r/L = 1$$

$$\tilde{\Psi} = |Psi| L^{3/2}$$


In [57]:
import numpy as np

# Let's try to put some numbers. If for example we have:
wr = 169  #Hz
wz = 26   #Hz
W  = 4*wr #Hz
ar = 1/np.sqrt(wr)
az = 1/np.sqrt(wz)

# characteristic quantities are:
T = 1/W
L = np.sqrt(2/wr)
E0 = W

#the adimensional quantities are:
W_adim = W*T   #adimensional Rabi frequency
gamma_r = wr*T #adimensional radial trap frequency
gamma_z = wz*T #adimensional axial trap frequency
sigma_r = ar/L #adimensional radial size
sigma_z = az/L #adimensional axial size

print("\nThe adimensional quantities are:\n")
print(f"W_A   = {W_adim}" f"; wr_A  = {gamma_r}" f"; wz_A  = {gamma_z}")
print(f"ar_A  = {sigma_r}" f"; az_A  = {sigma_z}")


The adimensional quantities are:

W_A   = 1.0; wr_A  = 0.25; wz_A  = 0.038461538461538464
ar_A  = 1.0; az_A  = 2.5495097567963922


With those choises for the characteristic lengths so the initial size of the BEC will be unitary, as well the Rabi frequency. 

The weak point in taking these definitions for L,T and E0, is that is the if $\tilde{\omega_r}$ becomes much smaller than 1, length scales in the simulation might become large, leading to increased computational costs.



# GPE adimensional coefficients
Given the equation 
$$i\hbar \partial_t \begin{pmatrix} \Psi_1 \\ \Psi_2 \end{pmatrix} = \left[
-\frac{\hbar^2}{2m} \Delta + V_{ext} + \hbar/2 \begin{pmatrix} \delta & \Omega \\ \Omega & -\delta \end{pmatrix} +
\begin{pmatrix} g_{11}|\Psi_1|^2+g_{12}|\Psi_2|^2 & 0 \\ 0 & g_{22}|\Psi_2|^2+g_{12}|\Psi_1|^2 \end{pmatrix}
\begin{pmatrix} \Psi_1 \\ \Psi_2 \end{pmatrix}
\right] $$

we can write the its adimensional version:
$$ i \partial_t \begin{pmatrix} \tilde{\Psi}_1 \\ \tilde{\Psi}_2 \end{pmatrix} = \left[
-\alpha \Delta + V_{ext} +
1/2 \begin{pmatrix} \delta T & \Omega T \\ \Omega T & -\delta T \end{pmatrix} +
 \begin{pmatrix} \gamma_{11}|\tilde{\Psi}_1|^2+\gamma_{12}|\tilde{\Psi}_2|^2 & 0 \\ 0 & \gamma_{22}|\tilde{\Psi}_2|^2+\gamma_{12}|\tilde{\Psi}_1|^2 \end{pmatrix}
\begin{pmatrix} \tilde{\Psi}_1 \\ \tilde{\Psi}_2 \end{pmatrix}
\right] $$

where
$$\alpha = \frac{1}{2} \frac{\hbar}{m} \frac{T}{L^2} \rightarrow  \frac{1}{2} \frac{T}{L^2} $$

$$\gamma_i =  \frac{N T}{\hbar L^3} \frac{4\pi\hbar^2 a_s}{m} \rightarrow \frac{N T}{L^3}  4\pi a_s $$

and $\Psi$ is normalized to 1.

Since we will simulate in a 2D domine, we have to rescale the density $n_{3D}$ to the $n_{1D}$.
$$n_{1D} = \frac{N}{\sqrt{2\pi}\sigma_z}$$ with $\sigma_z = R_{TF_z}/\sqrt{5} \rightarrow \sqrt{2/\omega_z}$ for a BEC with Thomas-Fermi profile and $\hbar=1$ and $m=1$. In addiction the adimensionalization of $\Psi$ is done defining $\tilde{\Psi}=\Psi L^2$. 

As a consequence we can write the adimnesional coefficient $\gamma$ for the 2D simulation:
$$\gamma_i =  \frac{n_{1D}g}{\hbar}\frac{T}{L^2} \rightarrow n_{1D}(4\pi a_s)\frac{T}{L^2}$$

In [3]:
import numpy as np

# Let's try to put some numbers. If for example we have:

# with our characteristic lengths:
a_bohr = 5.2917721e-11  # m Bohr radius
a11 = (85*a_bohr); 
a22 = (33.4*a_bohr);
a12 = (-53.0*a_bohr);
wr = 169*2*np.pi  #Hz
wz = 26*2*np.pi   #Hz
W  = 4*wr #Hz
ar = np.sqrt(1/wr)
az = np.sqrt(2/wz)

# characteristic quantities are:
T = 1/W
L = ar

n1D = 2e9

alpha = 0.5*(T/L**2)
gamma_11 = n1D*(4*np.pi*a11) * T/L**2
gamma_22 = n1D*(4*np.pi*a22) * T/L**2
gamma_12 = n1D*(4*np.pi*a12) * T/L**2

print("\nThe adimensional GPE coefficients are:\n")
print(f"alpha   = {alpha}")
print(f"gamma_11  = {gamma_11}" f"; gamma_22  = {gamma_22}" f"; gamma_12  = {gamma_12}")

print((gamma_11 + gamma_22 - 0.5*gamma_12)/4)


The adimensional GPE coefficients are:

alpha   = 0.125
gamma_11  = 28.261807001513432; gamma_22  = 11.105227692359396; gamma_12  = -17.62206789506132
12.044517160350871


## Ground state solution
### Initial guess for $\Psi$

I define the gaussian wavefunction profile as initial guess

## How to test the code

- I can do the first tests in looking at the spin composition as a function of density
- Once the simulation gives me a solution I can calculate the energy 
$$\langle{\Psi}|H|{\Psi}\rangle = \langle{\Psi}| \left(H_{\Omega} + \mu/2 + E_{kin} + E_{trap}\right)|\Psi\rangle$$

Without interactions (super dilute) the energy of the ground state is known $E_0 = -\frac{\Omega}{2}\sqrt{1+\delta^2/\Omega^2} + \hbar \Omega$ 

- Then one may calculate the difference in energy $\Delta E = 1/2 mv^2 $, and I will integrate $$\int \Delta E n_{1D} dz$$

## To think about:
- another way to rescale the g is simbìply to consider the 1D density instead of the n 3D $n_{3D} = n_{1D} |\Psi|^2$
 so if it is not gaussian my wavefunction I rescale the g by the density 1D. Another way to think abour it is that we replace $g_{--} \rightarrow g_{--}n$ 